In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch, cv2, random, os

In [ ]:
class serit_dataset(Dataset):
    def __init__(self, folder=0, transform=None, path_added=False):
        self.path = "/home/aye/robotaksi_main/main_ws/rb22_main/rb22_sim_serit/src/logger/data/" + str(folder)
        self.img_names = list()
        self.steer = list()
        self.secim = list()
        self.transform = transform
        self.path_added = path_added
        self.read_data(self.path + "/logger.csv")

    def read_data(self, folder):
        arr = np.genfromtxt(folder, delimiter=",", skip_header=1, dtype=None, encoding='UTF-8')
        for i in range(0, len(arr)):
            if not self.path_added:
                name = self.path + "/" + str(arr[i][0]) + ".jpg"
            else:
                name = str(arr[i][0])
            self.img_names.append(name)
            self.steer.append(round(arr[i][1], 2))
            self.secim.append(int(arr[i][2]))

        self.lenght = len(self.img_names)

    def __getitem__(self,index):
        if torch.is_tensor(index):
            index = index.tolist()

        out = self.img_names[index], self.steer[index], self.secim[index]
        if self.transform:
            out = self.transform(out)
        return out

    def __len__(self):
        return len(self.img_names)

    def pop(self, index):
        self.img_names.pop(index)
        self.steer.pop(index)
        self.secim.pop(index)

    def __setitem__(self,index,value):
        self.img_names[index], self.steer[index], self.secim[index] = value

    def save(self, filename):
        folder = "logger.csv"
        with open(folder, 'w') as f:
            writer = csv.writer(f)
            for i in range(len(self.img_names)):
                writer.writerow([self.img_names[i], self.steer[i], self.secim[i]])

class saved_dataset(Dataset):
    def __init__(self, transform=None):
        self.img_names = list()
        self.steer = list()
        self.secim = list()
        self.transform = transform
        self.read_data("logger.csv")

    def read_data(self, folder):
        arr = np.genfromtxt(folder, delimiter=",", dtype=None, encoding='UTF-8')
        for i in range(0, len(arr)):
            name = str(arr[i][0])
            self.img_names.append(name)
            self.steer.append(round(arr[i][1], 2))
            self.secim.append(int(arr[i][2]))

        self.lenght = len(self.img_names)

    def __getitem__(self,index):
        if torch.is_tensor(index):
            index = index.tolist()

        out = self.img_names[index], self.steer[index], self.secim[index]
        if self.transform:
            out = self.transform(out)
        return out

    def __len__(self):
        return len(self.img_names)

    def pop(self, index):
        self.img_names.pop(index)
        self.steer.pop(index)
        self.secim.pop(index)

    def __setitem__(self,index,value):
        self.img_names[index], self.steer[index], self.secim[index] = value

    def save(self, filename):
        folder = "logger.csv"
        with open(folder, 'w') as f:
            writer = csv.writer(f)
            for i in range(len(self.img_names)):
                writer.writerow([self.img_names[i], self.steer[i], self.secim[i]])
    
class ToTensor(object):
    def __call__(self, sample):
        name, steer, secim = sample
        img = cv2.imread(name)
        img = img[370:,:,:]
        img = img.transpose((2, 0, 1))
        steer = round((steer + 1) * 5, 2)
        return img, steer, secim

In [ ]:
dataset = serit_dataset(2, path_added=False)
img,_,_ = dataset[0]
image = Image.open(img)
plt.figure(figsize=(15,10))
plt.imshow(image)

In [ ]:
im = np.asarray(image)
plt.figure(figsize=(15,5))
plt.imshow(im[370:,:,:])

In [ ]:
img, _, _ = dataset[525]
im = Image.open(img)
im = np.asarray(im)
plt.figure(figsize=(15,10))
plt.imshow(im[370:,:,:])

In [ ]:
len(dataset)

In [ ]:
hist, bins = np.histogram(dataset.steer,39)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
hist, bins = np.histogram(dataset.secim,39)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
steer_duz = list()
steer_sol = list()
steer_sag = list()
for i in range(0, len(dataset)):
    if str(dataset.secim[i]) == '0':
        steer_duz.append(dataset.steer[i])
    elif str(dataset.secim[i]) == '2':
        steer_sol.append(dataset.steer[i])
    else:
        steer_sag.append(dataset.steer[i])

hist, bins = np.histogram(steer_duz,25)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
def try_to_balance(data, max_size):
    random.shuffle(dataset)
    duz = [0] * 201
    sol = [0] * 201
    sag = [0] * 201
    plist = list()
    for i in range(0, len(dataset)):
        if str(data.secim[i]) == '0':
            ind = int((data.steer[i] * 100) + 100)
            duz[ind] += 1

            if duz[ind] >= max_size:
                plist.append(i)
                print("will be drop from duz", i)

        if str(data.secim[i]) == '1':
            ind = int((data.steer[i] * 100) + 100)
            sag[ind] += 1

            if sag[ind] >= max_size:
                plist.append(i)
                print("will be drop from sag", i)
        
        if str(data.secim[i]) == '2':
            ind = (int((data.steer[i] * 100) + 100))
            sol[ind] += 1

            if sol[ind] >= max_size:
                plist.append(i)
                print("will be drop from sol", i)

    print("duz liste", duz)
    print("sag liste", sag)
    print("sol liste", sol)
    plist.sort(reverse=True)
    for i in range(0, len(plist)):
        data.pop(plist[i])

In [ ]:
try_to_balance(dataset, 3000)
steer_duz = list()
steer_sol = list()
steer_sag = list()
for i in range(0, len(dataset)):
    if str(dataset.secim[i]) == '0':
        steer_duz.append(dataset.steer[i])
    elif str(dataset.secim[i]) == '2':
        steer_sol.append(dataset.steer[i])
    else:
        steer_sag.append(dataset.steer[i])

In [ ]:
hist, bins = np.histogram(steer_duz,25)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
hist, bins = np.histogram(steer_sol,25)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
hist, bins = np.histogram(steer_sag,25)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.025)

In [ ]:
dataset.save("logger.csv")

In [ ]:
dataset_int_save = saved_dataset()

In [ ]:
dataset_int_save[0][1:]

In [ ]:
for i in range(len(dataset_int_save)):
    img_name, *_ = dataset_int_save[i]
    img = cv2.imread(img_name)
    img_name = img_name.split('/')[-1]
    img = img[370:,:,:]
    cv2.imwrite("/home/aye/serit_egitim/data/" + img_name, img)

In [ ]:
dataset_new = saved_dataset(transform=ToTensor())

In [ ]:
dataset_new[0][1:]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = (350,1280)
num_of_epoch = 40
batch_size = 8
learning_rate = 1e-4

In [ ]:
trsplit = int(len(dataset_new) * 0.8)
ttsplit = len(dataset_new) - trsplit
train, test = torch.utils.data.random_split(dataset_new, [trsplit, ttsplit])
oran = int(len(train) * 0.9)
train_subset, val_subset = torch.utils.data.random_split(train, [oran, len(train) - oran])

In [ ]:
train_loader = DataLoader(dataset=train_subset.dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=True)
val_loader = DataLoader(dataset=val_subset.dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=True)
test_loader = DataLoader(dataset=test.dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=True)
print(len(train_subset), len(val_subset), len(test_loader))

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, (5, 5), stride=(2, 2))
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(24, 36, (5, 5), stride=(2, 2))
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(36, 48, (3, 3), stride=(2, 2))
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(48, 64, (3, 3), stride=(1, 1))
        self.relu4 = nn.ReLU()

        self.pool = nn.AvgPool2d(40)

        self.fc1 = nn.Linear(1537, 384)
        self.fc2 = nn.Linear(384, 64)
        self.fc3 = nn.Linear(64, 1)


    def forward(self, x1, x2):
        out = self.conv1(x1)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.relu3(out)

        out = self.conv4(out)
        out = self.relu4(out)

        out = self.pool(out)
        
        out = out.view(out.size(0),-1)
        out = torch.cat((out, x2), 1)

        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

In [ ]:
model = ConvNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_steps = len(train_loader)
for epoch in range(num_of_epoch):

    for i, (img,angle,secim) in enumerate(train_loader):
        img = img.to(device)
        angle = angle.to(device)
        x = secim.detach().cpu().numpy()
        x = np.array([x]).reshape(x.shape[0],1)
        secim = torch.from_numpy(x)
        secim = secim.to(device)

        img = img.float()
        angle = angle.float()
        secim = secim.float()

        outputs = model(img, secim)
        outputs = outputs.reshape(-1)

        loss = criterion(outputs, angle)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_of_epoch}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')

    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    for i, (img,angle,secim) in enumerate(val_loader, 0):
        with torch.no_grad():
            img = img.to(device)
            angle = angle.to(device)

            x = secim.detach().cpu().numpy()
            x = np.array([x]).reshape(x.shape[0],1)
            secim = torch.from_numpy(x)
            secim = secim.to(device)

            img = img.float()
            angle = angle.float()
            secim = secim.float()

            outputs = model(img, secim)
            
            outputs = outputs.detach().cpu().numpy()
            outputs = np.round(outputs)
            outputs = torch.from_numpy(outputs).to(device)
            
            total += outputs.size(0)
            correct += (outputs == angle).sum().item()

            outputs = outputs.reshape(-1)
            loss = criterion(outputs, angle)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    print("acc = ", 100 * (correct / total))
    print("loss = ", val_loss / val_steps)

path = "./model_sgd3.pth"
torch.save(model.state_dict(), path)

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (images, labels, selections) in enumerate(test_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        x = selections.detach().cpu().numpy()
        x = np.array([x]).reshape(x.shape[0],1)
        selections = torch.from_numpy(x)
        selections = selections.to(device)


        images = images.float()
        labels = labels.float()
        selections = selections.float()

        outputs = model(images, selections)

        outputs = outputs.detach().cpu().numpy()
        outputs = np.round(outputs, 2)
        outputs = torch.from_numpy(outputs).to(device)

        n_samples += labels.size(0)
        n_correct += (outputs == labels).sum().item()
        
    acc = 100.0 * (n_correct / n_samples)
    print(f'Accuracy of the network: {acc} %')
    print(n_correct, n_samples)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1,kernel_size=3,padding=1,bias=False):
        super(ResidualBlock,self).__init__()
        self.cnn1 =nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(out_channels,out_channels,kernel_size,1,padding,bias=False),
            nn.BatchNorm2d(out_channels)
        )
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
            
    def forward(self,x):
        residual = x
        x = self.cnn1(x)
        x = self.cnn2(x)
        x += self.shortcut(residual)
        x = nn.ReLU(True)(x)
        return x

class Res18Model(nn.Module):
    def __init__(self):
        super(Res18Model, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )

        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AvgPool2d(12)

        self.fc1 = nn.Linear(1537, 384)
        self.fc2 = nn.Linear(384, 64)
        self.fc3 = nn.Linear(64, 1)


    def forward(self,x1, x2):

        out = self.block1(x1)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)

        out = self.avgpool(out)

        out = out.view(out.size(0),-1)
        out = torch.cat((out, x2), 1)

        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out
        

In [ ]:
model = Res18Model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4e-4, momentum=0.1)
criterion = nn.MSELoss()

In [ ]:
total_steps = len(train_loader)
for epoch in range(num_of_epoch):
    running_acc = 0
    for i, (img,angle,secim) in enumerate(train_loader):
        img = img.to(device)
        angle = angle.to(device)
        x = secim.detach().cpu().numpy()
        x = np.array([x]).reshape(x.shape[0],1)
        secim = torch.from_numpy(x)
        secim = secim.to(device)

        img = img.float()
        angle = angle.float()
        secim = secim.float()

        optimizer.zero_grad()
        outputs = model(img, secim)

        outputs = outputs.reshape(-1)
        loss = criterion(outputs, angle)
        running_acc += (outputs == angle).float().mean()

        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_of_epoch}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}, accuracy: [{running_acc.item()}]')

    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    for i, (img,angle,secim) in enumerate(val_loader, 0):
        with torch.no_grad():
            img = img.to(device)
            angle = angle.to(device)

            x = secim.detach().cpu().numpy()
            x = np.array([x]).reshape(x.shape[0],1)
            secim = torch.from_numpy(x)
            secim = secim.to(device)

            img = img.float()
            angle = angle.float()
            secim = secim.float()

            outputs = model(img, secim)
                        
            total += outputs.size(0)
            correct += (outputs == angle).sum().item()

            outputs = outputs.reshape(-1)
            loss = criterion(outputs, angle)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    print("acc = ", 100 * (correct / total))
    print("loss = ", val_loss / val_steps)

path = "./modelres18-2.pth"
torch.save(model.state_dict(), path)

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (images, labels, selections) in enumerate(test_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        x = selections.detach().cpu().numpy()
        x = np.array([x]).reshape(x.shape[0],1)
        selections = torch.from_numpy(x)
        selections = selections.to(device)


        images = images.float()
        labels = labels.float()
        selections = selections.float()

        outputs = model(images, selections)

        outputs = outputs.detach().cpu().numpy()
        outputs = np.round(outputs, 2)
        outputs = torch.from_numpy(outputs).to(device)

        n_samples += labels.size(0)
        n_correct += (outputs == labels).sum().item()
        
    acc = 100.0 * (n_correct / n_samples)
    print(f'Accuracy of the network: {acc} %')
    print(n_correct, n_samples)